In [1]:
# Single Dummy Solver. Input is a PBN deal. Output is a DataFrame.

!pip install pandas --quiet
!pip install endplay --quiet

import pandas as pd

from endplay.dealer import generate_deals
from endplay.types import Deal
from endplay.dds import calc_all_tables

In [2]:
def calc_double_dummy_deals(deals, batch_size=40):
    t_t = []
    tables = []
    for b in range(0,len(deals),batch_size):
        batch_tables = calc_all_tables(deals[b:min(b+batch_size,len(deals))])
        tables.extend(batch_tables)
        batch_t_t = (tt._data.resTable for tt in batch_tables)
        t_t.extend(batch_t_t)
    assert len(t_t) == len(tables)
    return deals, t_t, tables

In [3]:
def constraints(deal):
    return True


def generate_single_dummy_deals(predeal_string, produce, env=dict(), max_attempts=1000000, seed=None, show_progress=True, strict=True, swapping=0):
    
    predeal = Deal(predeal_string)

    deals_t = generate_deals(
        constraints,
        predeal=predeal,
        swapping=swapping,
        show_progress=show_progress,
        produce=produce,
        seed=seed,
        max_attempts=max_attempts,
        env=env,
        strict=strict
        )

    deals = tuple(deals_t) # create a tuple before interop memory goes wonky
    
    return calc_double_dummy_deals(deals)

In [4]:
def calculate_single_dummy_probabilities(deal, produce=100):

    ns_ew_rows = []
    for ns_ew in ['NS','EW']:
        s = deal[2:].split()
        if ns_ew == 'NS':
            s[1] = '...'
            s[3] = '...'
        else:
            s[0] = '...'
            s[2] = '...'
        predeal_string = 'N:'+' '.join(s)
        #print(f"predeal:{predeal_string}")

        d_t, t_t, tables = generate_single_dummy_deals(predeal_string, produce, show_progress=False)

        rows = []
        max_display = 4 # pprint only the first n generated deals
        direction_order = [0,2,3,1]
        suit_order = [3,2,1,0,4]
        for ii,(dd,sd,tt) in enumerate(zip(d_t,t_t,tables)):
            if ii < max_display:
                print(f"Deal:{ii+1} Fixed:{ns_ew} Generated:{ii+1}/{produce}")
                dd.pprint()
                print()
                tt.pprint()
                print()
            nswe_flat_l = [sd[suit][direction] for direction in direction_order for suit in suit_order]
            rows.append([dd.to_pbn()]+nswe_flat_l)

        dd_df = pd.DataFrame(rows,columns=['Deal']+[d+s for d in list('NSWE') for s in list('CDHSN')])
        for d in list('NSWE'):
            for s in list('SHDCN'):
                ns_ew_rows.append([ns_ew,d,s]+dd_df[d+s].value_counts(normalize=True).reindex(range(14), fill_value=0).tolist())
    
    df = pd.DataFrame(ns_ew_rows,columns=['Fixed_Direction','Declarer','Suit']+['Take_'+str(n) for n in range(14)])
    return df

In [5]:
#endplay._dds.SetMaxThreads(24)

In [6]:
# show double dummy results.
pbn_deals = ['N:A98.Q43.J.AKQJ93 KT.AJ8.QT9753.87 QJ73.9752.A64.T6 6542.KT6.K82.542']
deals = [Deal(pbn_deal) for pbn_deal in pbn_deals] # convert pbns to deals
deals, t_t, tables = calc_double_dummy_deals(deals)

# all deals are calculated. print formatted deals, formatted dd tables, create dataframe of all 20 (directions x strains) dd values.
rows = []
direction_order = [0,2,3,1]
suit_order = [3,2,1,0,4]
for ii,(dd,sd,tt) in enumerate(zip(deals,t_t,tables)):
    dd.pprint()
    print()
    tt.pprint()
    print()
    nswe_flat_l = [sd[suit][direction] for direction in direction_order for suit in suit_order]
    rows.append([dd.to_pbn()]+nswe_flat_l)
df = pd.DataFrame(rows,columns=['Deal']+[d+s for d in list('NSWE') for s in list('CDHSN')])
df

              A98
              Q43
              J
              AKQJ93
6542                        KT
KT6                         AJ8
K82                         QT9753
542                         87
              QJ73
              9752
              A64
              T6

     ♣  ♦  ♥  ♠ NT
  N  9  4  8  9  8
  S  9  4  8  9  8
  E  4  9  5  4  4
  W  4  9  5  4  4



,Deal,NC,ND,NH,NS,NN,SC,SD,SH,SS,...,WC,WD,WH,WS,WN,EC,ED,EH,ES,EN
0,N:A98.Q43.J.AKQJ93 KT.AJ8.QT9753.87 QJ73.9752....,9,4,8,9,8,9,4,8,9,...,4,9,5,4,4,4,9,5,4,4


In [7]:
# show single dummy results. produce means generate 100 trials.
pbn_deal = 'N:A98.Q43.J.AKQJ93 KT.AJ8.QT9753.87 QJ73.9752.A64.T6 6542.KT6.K82.542'
produce = 100 # where 100 is number of single dummy samples to generate
df = calculate_single_dummy_probabilities(pbn_deal, produce)
display(f"Single dummy trick taking probs after {produce} samples {pbn_deal}",df)

Deal:1 Fixed:NS Generated:1/100
              A98
              Q43
              J
              AKQJ93
T5                          K642
J6                          AKT8
T9732                       KQ85
8752                        4
              QJ73
              9752
              A64
              T6

     ♣  ♦  ♥  ♠ NT
  N  9  3  8  8  8
  S  9  3  7  8  8
  E  3  9  5  3  3
  W  3  9  5  3  3

Deal:2 Fixed:NS Generated:2/100
              A98
              Q43
              J
              AKQJ93
K6                          T542
KT8                         AJ6
QT8532                      K97
85                          742
              QJ73
              9752
              A64
              T6

     ♣  ♦  ♥  ♠ NT
  N 10  5  9 10 10
  S 10  5  9 10 10
  E  2  8  4  2  2
  W  2  8  4  2  2

Deal:3 Fixed:NS Generated:3/100
              A98
              Q43
              J
              AKQJ93
542                         KT6
KJ8                         AT6
KT9875                 

'Single dummy trick taking probs after 100 samples N:A98.Q43.J.AKQJ93 KT.AJ8.QT9753.87 QJ73.9752.A64.T6 6542.KT6.K82.542'

,Fixed_Direction,Declarer,Suit,Take_0,Take_1,Take_2,Take_3,Take_4,Take_5,Take_6,Take_7,Take_8,Take_9,Take_10,Take_11,Take_12,Take_13
0,NS,N,S,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.12,0.33,0.48,0.04,0.00,0.00,0.0
1,NS,N,H,0.00,0.00,0.00,0.00,0.00,0.01,0.05,0.10,0.70,0.14,0.00,0.00,0.00,0.0
2,NS,N,D,0.00,0.00,0.05,0.11,0.43,0.36,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3,NS,N,C,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.70,0.30,0.00,0.00,0.0
4,NS,N,N,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.40,0.08,0.02,0.00,0.0
5,NS,S,S,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.16,0.36,0.42,0.04,0.00,0.00,0.0
6,NS,S,H,0.00,0.00,0.00,0.01,0.01,0.01,0.07,0.12,0.65,0.13,0.00,0.00,0.00,0.0
7,NS,S,D,0.00,0.00,0.05,0.16,0.40,0.34,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.0
8,NS,S,C,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.77,0.23,0.00,0.00,0.0
9,NS,S,N,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.53,0.38,0.07,0.02,0.00,0.0
